## 간단한 웹 크롤링?

### 아무런 지식이 없을 경우

In [51]:

from collections import defaultdict

def get_txt(txt, start, end) : 
    cnt = txt.find(start)
    txt = txt[cnt + len(start) :]
    cnt =txt.find(end)
   
    return txt[ : cnt]

def get_link(txt, start, end) :
    stack = []
    while txt.find(start) != -1 :
        cnt = txt.find(start)
        txt = txt[cnt + len(start) :]
        cnt =txt.find(end)
        
        stack.append(txt[ : cnt])
        txt = txt[len(end) : ]
    return stack


def get_score(txt, keyword) :
     
    txt = txt.lower()
    keyword = keyword.lower()
    score = 0.0
    
    while txt.find(keyword) != -1 :
        cnt = txt.find(keyword)
        
        a = txt[cnt-1]
        b = txt[cnt + len(keyword)]
        
        if a.isalpha() == False and b.isalpha() == False :
            score +=1
        
        txt = txt[cnt + len(keyword) +1:]
    return score


    
def solution(word, pages):
    answer = []
    temp = []
    dic_link = defaultdict(list)
    dic_score = defaultdict(list)
    dic_link_score = defaultdict(list)
    
    for d in pages :
              
        name = get_txt(d, start = '<meta property=\"og:url\" content=\"', end = '\"')
        temp.append(name)
        txt = get_txt(d, start = '<body>', end = '</body>')
        dic_link[name] += get_link(txt,start = 'a href=\"', end = '\"')
        dic_score[name] = get_score(txt,word) 

    
    for keys in temp :
        dic_link_score[keys] = 0.0
        
    for keys in dic_score.keys() :
        score = dic_score[keys]
        for v in dic_link[keys] : 
            total = len(dic_link[keys])   
            if v in dic_link_score :
         
                dic_link_score[v] += score/total
            
    n_max = -1
    for idx, v in enumerate(temp) :
        if dic_link_score[v] + dic_score[v] > n_max :
            answer = idx
            n_max = dic_link_score[v] + dic_score[v]

  
    print(dic_link)
    print(dic_score)
    print( dic_link_score)
   
    
    return answer

### 문자열과 조금의 정규표현식 응용한 경우

In [286]:
# 정규식 테스트 사이트 : https://regexr.com
# 정규식 시각 테스트 사이트 : https://regexper.com/#%3Ctr.*%3F%3E%28.*%3F%29%3C%5C%2Ftr%3E

In [317]:
import re

def solution(word, pages):
    webpage = {} # [{idx, link_stack, score, total_score }
    
    
    for idx, page in enumerate(pages) :
        page_name = page.split('<meta property=\"og:url\" content=\"')[1].split('\"')[0]
        
        link = []
        for link_data in page.split('<a href=\"')[1:] : 
            link.append(link_data.split('\"')[0])
         
        score = re.findall('[a-z]+',page.lower()).count(word.lower())
        
        webpage[page_name] = [idx, link,score, score]
    
    for keys in webpage.keys() :
        idx, link_stack, score, _ = webpage[keys]
        for link in link_stack : 
            if  link in webpage :
                webpage[link][3] += score/len(link_stack)
    
    n_max = -1
    
    for keys  in webpage.keys() :
        idx, _, _, total_score = webpage[keys]
        if total_score > n_max : 
            n_max = total_score
            answer = idx
    
    print(webpage)
    
    return answer
    
    value":(.*?)}
    

### 정규표현식 버전

In [392]:
import re

def solution(word, pages):
    webpage = {} # {idx, links, score, total_score }
    
    for idx, page in enumerate(pages) :
        page_name = re.findall('<meta property=\\"og:url\\" content=\\"(.*?)"/>',page)[0]
        links = re.findall('<a href=\\"(.*?)">',page)
        score = re.findall('[a-z]+',page.lower()).count(word.lower())
        total_score = score
        webpage[page_name] = [idx, links, score, total_score]
     
    for key in webpage.keys() :
        _, links, score, _ = webpage[key]
        for link in links : 
            if  link in webpage :
                webpage[link][3] += score/len(links)
    
    answer = []
    for idx, _, _, total_score in webpage.values() :
         answer.append([idx,total_score])
    
    answer.sort(key = lambda x : (-x[1],x[0]))
    for key in webpage.keys() :
        print(key,' : ',webpage[key])
    
    return answer[0][0]
    
    
    

In [393]:
# 답 0 
solution("blind",["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://a.com\"/>\n</head>  \n<body>\nBlind Lorem Blind ipsum dolor Blind test sit amet, consectetur adipiscing elit. \n<a href=\"https://b.com\"> Link to b </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://b.com\"/>\n</head>  \n<body>\nSuspendisse potenti. Vivamus venenatis tellus non turpis bibendum, \n<a href=\"https://a.com\"> Link to a </a>\nblind sed congue urna varius. Suspendisse feugiat nisl ligula, quis malesuada felis hendrerit ut.\n<a href=\"https://c.com\"> Link to c </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://c.com\"/>\n</head>  \n<body>\nUt condimentum urna at felis sodales rutrum. Sed dapibus cursus diam, non interdum nulla tempor nec. Phasellus rutrum enim at orci consectetu blind\n<a href=\"https://a.com\"> Link to a </a>\n</body>\n</html>"])

[[0, 4.5], [1, 4.0], [2, 1.5]]
https://a.com  :  [0, ['https://b.com'], 3, 4.5]
https://b.com  :  [1, ['https://a.com', 'https://c.com'], 1, 4.0]
https://c.com  :  [2, ['https://a.com'], 1, 1.5]
[[0, 4.5], [1, 4.0], [2, 1.5]]


0

In [394]:
# 답 1
solution("Muzi",["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://careers.kakao.com/interview/list\"/>\n</head>  \n<body>\n<a href=\"https://programmers.co.kr/learn/courses/4673\"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://www.kakaocorp.com\"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href=\"https://hashcode.co.kr/tos\"></a>\n\n\t^\n</body>\n</html>"])

[[0, 0], [1, 1]]
https://careers.kakao.com/interview/list  :  [0, ['https://programmers.co.kr/learn/courses/4673'], 0, 0]
https://www.kakaocorp.com  :  [1, ['https://hashcode.co.kr/tos'], 1, 1]
[[1, 1], [0, 0]]


1

In [342]:
def solution(word, pages):
    answer = 0    
    for idx, page in enumerate(pages) :
        print(page)
    return answer

In [320]:
data = '<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://careers.kakao.com/interview/list\"/>\n</head>  \n<body>\n<a href=\"https://programmers.co.kr/learn/courses/4673\"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://www.kakaocorp.com\"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href=\"https://hashcode.co.kr/tos\"></a>\n\n\t^\n</body>\n</html>'

In [ ]:
import re
p = re.compile('<meta property=\\"og:url\\" content=\\"(.*?)"/>')

data

# 문자열 범위 표현방법
- 범위로 표현하면 길이에 대한 예외처리를 할 필요가 없다

In [1]:
s = 'string'

In [2]:
s[-1]

'g'

In [6]:
s[5:6]

'g'

In [4]:
s[6]

IndexError: string index out of range

In [5]:
s[6:7]

''

fro??
????o
fr???
fro???
pro?
[3, 2, 4, 1, 0]


[3, 2, 4, 1, 0]